In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import xarray as xr
import numpy as np
import geopandas as gp

from unseen import fileio
from unseen import time_utils

## Observations

In [17]:
agcd_file = "/g/data/xv83/dbi599/ag/data/pr_agcd_1900-2019_A-DEC-mean_wheat-sheep-mean.zarr.zip"

In [18]:
agcd_ds = fileio.open_file(agcd_file)

In [19]:
agcd_ds

<xarray.Dataset>
Dimensions:  (region: 4, time: 121)
Coordinates:
  * region   (region) <U10 'south-west' 'south-east' 'north-east' 'all'
  * time     (time) object 1900-12-31 00:00:00 ... 2020-12-31 00:00:00
Data variables:
    pr       (time, region) float64 dask.array<chunksize=(1, 4), meta=np.ndarray>
Attributes: (12/29)
    Conventions:               CF-1.6, ACDD-1.3
    acknowledgment:            The Australian Government, Bureau of Meteorolo...
    agcd_version:              AGCD v2.0.0 Snapshot (1900-01-01 to 2020-05-31)
    analysis_components:       total: the gridded accumulation of rainfall.
    attribution:               Data should be cited as : Australian Bureau of...
    cdm_data_type:             Grid
    ...                        ...
    summary:                   The monthly rainfall data represents the amoun...
    time_coverage_end:         1900-12-31T00:00:00
    time_coverage_start:       1900-01-01T00:00:00
    title:                     Interpolated Rain Gauge Precipitation
    url:                       http://www.bom.gov.au/climate/
    uuid:                      43596dc1-c56e-42a2-ba87-4e3b726a6e60

In [20]:
agcd_ds['pr'] = agcd_ds['pr'] * 365
agcd_ds['pr'].attrs['units'] = 'mm yr-1'

In [21]:
years = agcd_ds['time'].dt.year.values
agcd_df = pd.DataFrame(index=years)
agcd_df['all'] = agcd_ds['pr'].sel(region='all').values
agcd_df['south-west'] = agcd_ds['pr'].sel(region='south-west').values
agcd_df['south-east'] = agcd_ds['pr'].sel(region='south-east').values
agcd_df['north-east'] = agcd_ds['pr'].sel(region='north-east').values

In [24]:
agcd_df

,all,south-west,south-east,north-east
1900,464.811006,475.969258,364.933123,527.053421
1901,400.237197,350.454185,303.321759,504.458390
1902,311.070922,375.835685,241.523708,313.200854
1903,545.881173,426.354388,402.868296,732.982318
1904,497.322876,504.010219,337.707338,604.851234
...,...,...,...,...
2016,580.077608,463.919797,533.637226,696.759149
2017,470.199229,432.314060,365.637155,571.200521
2018,357.549319,377.360680,240.977893,425.286678
2019,246.779661,265.421957,217.078133,254.208977


In [25]:
agcd_df.quantile(0.33)

all           451.652868
south-west    382.220147
south-east    338.208430
north-east    551.451692
Name: 0.33, dtype: float64

In [26]:
agcd_df.quantile(0.66)

all           520.235589
south-west    436.211417
south-east    401.178648
north-east    670.629388
Name: 0.66, dtype: float64

In [42]:
agcd_terciles_df = pd.DataFrame(index=years)
agcd_terciles_df['south-east'] = pd.qcut(agcd_df['south-east'], q=3, labels=['dry', 'normal', 'wet'])
agcd_terciles_df['south-west'] = pd.qcut(agcd_df['south-west'], q=3, labels=['dry', 'normal', 'wet'])
agcd_terciles_df['north-east'] = pd.qcut(agcd_df['north-east'], q=3, labels=['dry', 'normal', 'wet'])

In [43]:
agcd_terciles_df[-30:]

,south-east,south-west,north-east
1991,dry,dry,normal
1992,wet,wet,normal
1993,wet,normal,dry
1994,dry,dry,dry
1995,wet,wet,normal
1996,normal,normal,wet
1997,dry,normal,normal
1998,normal,normal,wet
1999,wet,wet,wet
2000,wet,wet,normal


In [44]:
agcd_terciles_df.groupby(['south-west', 'south-east', 'north-east']).size()

south-west  south-east  north-east
dry         dry         dry           15
                        normal         5
                        wet            1
            normal      dry            2
                        normal         7
                        wet            3
            wet         dry            0
                        normal         3
                        wet            4
normal      dry         dry            7
                        normal         4
                        wet            2
            normal      dry            3
                        normal        10
                        wet            3
            wet         dry            3
                        normal         1
                        wet            7
wet         dry         dry            3
                        normal         3
                        wet            0
            normal      dry            7
                        normal         1
                      

In [80]:
(agcd_terciles_df.groupby(['south-west', 'south-east', 'north-east']).size() / 120) * 100

south-west  south-east  north-east
dry         dry         dry           12.500000
                        normal         4.166667
                        wet            0.833333
            normal      dry            1.666667
                        normal         5.833333
                        wet            2.500000
            wet         dry            0.000000
                        normal         2.500000
                        wet            3.333333
normal      dry         dry            5.833333
                        normal         3.333333
                        wet            1.666667
            normal      dry            2.500000
                        normal         8.333333
                        wet            2.500000
            wet         dry            2.500000
                        normal         0.833333
                        wet            5.833333
wet         dry         dry            2.500000
                        normal         2.500000
     

In [47]:
all_dry = (agcd_terciles_df['south-west'] == 'dry') & (agcd_terciles_df['south-east'] == 'dry') & (agcd_terciles_df['north-east'] == 'dry')

In [52]:
agcd_terciles_df[all_dry].index.values

array([1901, 1902, 1922, 1937, 1940, 1944, 1948, 1957, 1972, 1980, 1994,
       2002, 2009, 2018, 2019])

In [53]:
all_wet = (agcd_terciles_df['south-west'] == 'wet') & (agcd_terciles_df['south-east'] == 'wet') & (agcd_terciles_df['north-east'] == 'wet')

In [54]:
agcd_terciles_df[all_wet].index.values

array([1917, 1920, 1921, 1931, 1955, 1958, 1963, 1971, 1973, 1974, 1975,
       1978, 1988, 1999, 2011, 2016])

## Forecast data

In [55]:
cafe_bc_file = "/g/data/xv83/dbi599/ag/data/pr_cafe-c5-d60-pX-f6_19900501-20191101_A-DEC-mean_wheat-sheep-mean_bias-corrected-agcd-additive.zarr.zip"

In [56]:
cafe_bc_ds = fileio.open_file(cafe_bc_file)

In [57]:
cafe_bc_ds

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 78, lead_time: 9, region: 4)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1982-11-01 00:00:00 ... 2021-11-01 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8
  * region     (region) <U10 'south-west' 'south-east' 'north-east' 'all'
    time       (lead_time, init_date) object dask.array<chunksize=(9, 78), meta=np.ndarray>
Data variables:
    pr         (init_date, lead_time, ensemble, region) float64 dask.array<chunksize=(1, 9, 96, 4), meta=np.ndarray>
Attributes:
    history:  Wed Nov 03 13:54:17 2021: /g/data/xv83/dbi599/miniconda3/envs/u...

In [58]:
cafe_bc_ds['pr'] = cafe_bc_ds['pr'] * 365
cafe_bc_ds['pr'].attrs['units'] = 'mm yr-1'

In [64]:
cafe_samples = cafe_bc_ds['pr'].sel(lead_time=slice(3, None)).stack({'sample': ['ensemble', 'init_date', 'lead_time']})

In [81]:
nsamples = cafe_samples['sample'].shape[0]

In [82]:
nsamples

44928

In [73]:
np.arange(5) + 1

array([1, 2, 3, 4, 5])

In [74]:
samples = np.arange(nsamples) + 1
cafe_df = pd.DataFrame(index=samples)
cafe_df['south-west'] = cafe_samples.sel(region='south-west').values
cafe_df['south-east'] = cafe_samples.sel(region='south-east').values
cafe_df['north-east'] = cafe_samples.sel(region='north-east').values

In [75]:
cafe_df

,south-west,south-east,north-east
1,493.532015,377.322074,746.447210
2,338.689329,372.387079,600.961975
3,403.649736,463.406774,613.006862
4,516.517823,357.344918,510.338053
5,553.083644,203.554410,363.117858
...,...,...,...
44924,470.936329,248.324052,493.060030
44925,348.257607,275.233595,579.144019
44926,427.983150,307.856288,521.903277
44927,338.333944,535.636715,697.629422


In [76]:
cafe_terciles_df = pd.DataFrame(index=samples)
cafe_terciles_df['south-east'] = pd.qcut(cafe_df['south-east'], q=3, labels=['dry', 'normal', 'wet'])
cafe_terciles_df['south-west'] = pd.qcut(cafe_df['south-west'], q=3, labels=['dry', 'normal', 'wet'])
cafe_terciles_df['north-east'] = pd.qcut(cafe_df['north-east'], q=3, labels=['dry', 'normal', 'wet'])

In [77]:
cafe_terciles_df

,south-east,south-west,north-east
1,normal,wet,wet
2,normal,dry,normal
3,wet,normal,normal
4,normal,wet,normal
5,dry,wet,dry
...,...,...,...
44924,dry,wet,dry
44925,dry,dry,normal
44926,dry,normal,normal
44927,wet,dry,wet


In [83]:
(cafe_terciles_df.groupby(['south-west', 'south-east', 'north-east']).size() / nsamples) * 100

south-west  south-east  north-east
dry         dry         dry           7.391827
                        normal        4.017539
                        wet           1.531339
            normal      dry           3.311966
                        normal        4.985755
                        wet           3.207354
            wet         dry           0.787927
                        normal        2.708778
                        wet           5.390848
normal      dry         dry           6.704060
                        normal        3.374288
                        wet           1.048344
            normal      dry           3.583511
                        normal        4.727564
                        wet           2.946937
            wet         dry           1.006054
                        normal        3.240741
                        wet           6.701834
wet         dry         dry           5.889423
                        normal        2.644231
                        w